In [ ]:
pip install fuzzywuzzy

In [ ]:
pip install python-docx

     |████████████████████████████████| 5.6 MB 29.4 MB/s 
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184508 sha256=c4fa27d99a85fc5f83da7d3132ab3d8b7fa4fed9ff8a9800be2f37f14869e940
  Stored in directory: /root/.cache/pip/wheels/f6/6f/b9/d798122a8b55b74ad30b5f52b01482169b445fbb84a11797a6
Successfully built python-docx


In [ ]:
import nltk

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import re
import pandas as pd
import spacy
from spacy.util import filter_spans
from spacy.tokens import Span
from spacy.matcher import Matcher
from spacy import displacy

import docx
import json
import string
import numpy as np
from IPython.display import HTML, display
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

from nltk import tokenize
from operator import itemgetter
import math

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
import difflib


nlp = spacy.load("en_core_web_sm")
stop_words = nlp.Defaults.stop_words
exclude_item_list=['China', 'London', 'US', 'Us', 'United Kingdom', 'Singapore', 'year', 'years', 'work', 'preffered']

/usr/local/lib/python3.7/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
def main(filename):

    

    document = docx.Document(filename)
    docText = '\n\n'.join(
        paragraph.text for paragraph in document.paragraphs
    )
    txt = docText
        # #  Soft Skill
    lst = soft_skill_patterns(txt)
    lst = list(set(lst))

    #opeing the soft skills file
    f = open('soft_skills.txt', 'r+',encoding="utf-8")
    skills_list = f.read().splitlines()


    temp_list=[]
    for item in lst:
        #print(item)
        chk=difflib.get_close_matches(item, skills_list, n=1, cutoff=0.7)
        if len(chk)>0 and item not in temp_list:
            temp_list.append(item)

            
    soft_skill=validate_list_by_tfidf(txt,temp_list)
    print("--------Soft Skills-----------")
    print(set(soft_skill))
    
    
    # Hard Skill

    hard_lst=hard_skill_patterns(txt)

    # Extracting Experiences
    experience = extract_experience(txt)

    for item in experience:
        if item not in hard_lst:
            hard_lst.append(item)

    hard_lst=set(hard_lst)
    mylist=sorted(list(hard_lst), key=len, reverse=True)
    temp_list=mylist

    
    remove_item=[]
    for i in range(len(mylist)):
        for j in range(len(mylist)):
            if i!=j:
                
                if mylist[i] in mylist[j]:
                    #print(mylist[i], '   ', mylist[j])
                    #print(fuzz.ratio(mylist[i], mylist[j]))
                    if fuzz.ratio(mylist[i], mylist[j]) > 70:
                        if mylist[i] not in remove_item:
                            remove_item.append(mylist[i])

    for i in remove_item:
        if i in temp_list:
            temp_list.remove(i)

    #print(temp_list)
    
    
    
    f = open('hard_skills.txt', 'r+')
    skills_list = f.read().splitlines()
    skills_list=list(set(skills_list))
    
    t1=list(set(temp_list))

    chk=[]
    for i in range(len(skills_list)):
        for j in range(len(t1)):   
            #print(skills_list[i])

            if skills_list[i].lower() in t1[j].lower():
                #print(skills_list[i], '   ', t1[j])
                #print(fuzz.ratio(skills_list[i].lower(), t1[j].lower()))
                if fuzz.ratio(skills_list[i].lower(), t1[j].lower()) > 70:
                    #print(skills_list[i], '   ', t1[j])
                    #print(fuzz.ratio(skills_list[i].lower(), t1[j].lower()))
                    if t1[j] not in chk:
                        chk.append(t1[j])
    
    updated=chk.copy()
    #print(chk)
    for i in range(len(chk)):
        for j in range(i,len(chk)):
            if i!=j:
                #print(chk[i], '   ', chk[j])
                #print(fuzz.ratio(chk[i], chk[j]))
                    
                if fuzz.ratio(chk[i], chk[j]) > 70:
                    #print('test')
                    #print(chk[i], '   ', chk[j])
                    #print(fuzz.ratio(chk[i], chk[j]))
                    item = chk[j]
                    if chk[j] in updated:
                        #print(item)
                        #print(len(chk))
                        updated.remove(item)
                        #print(len(chk))
                        #print(len(updated))
        
    
    #print(updated)
    
    hard_skill_list=updated
    ''' 
    hard_skill_list=list(map(lambda x: x.replace('',''),hard_skill_list))
    for item in soft_skill:
        if item.lower() in hard_skill_list:
            print(item)
    
    '''   
    print("--------Hard Skills-----------")
    print(set(hard_skill_list))
    
    
    data={
        "Soft_skils":soft_skill,
        "Hard_skills":hard_skill_list
    }
    
    temp=json.dumps(data, indent = 4) 
    temp=json.loads(temp)
    all_skills=json.dumps(temp, indent=4)
    #print(json.dumps(temp, indent=4))
    return all_skills

In [ ]:
def clean_text(raw):
    try:
        # remove carriage returns and new lines
        raw = raw.replace('\r', '')
        raw = raw.replace('\n', ' ')
        
        # brackets appear in all instances
        raw = raw.replace('[', '')
        raw = raw.replace(']', '')
        raw = raw.replace(')', '')
        raw = raw.replace('(', '')
        
        # removing html tags
        clean_html = re.compile('<.*?>')
        clean_text = re.sub(clean_html, ' ', raw)
        
        # removing duplicate whitespace in between words
        clean_text = re.sub(" +", " ", clean_text) 
        
        # stripping first and last white space 
        clean_text = clean_text.strip()
        
        # commas had multiple spaces before and after in each instance
        clean_text = re.sub(" , ", ", ", clean_text) 
        
        # eliminating the extra comma after a period
        clean_text = clean_text.replace('.,', '.')        
        
        clean_text = re.sub('-', '', clean_text)
        clean_text = re.sub('_', '', clean_text)
        clean_text = re.sub('\[[^]]*\]', '', clean_text)
        #clean_text = re.sub(r'\n',' ',clean_text)
        clean_text = re.sub(r'[^\w\s]',' ',clean_text)
        
        clean_text = re.sub(r'[0-9]+','',clean_text)
        clean_text = clean_text.replace(u'\xa0', u' ')
        
    except:
        clean_text = np.nan
        
    return clean_text


def extract_experience(text):
    experience_word_list=[]
    
    test=re.findall(r'(experience[^.]*)', text,flags=re.IGNORECASE)
    test = ' '.join(test)
    test=nlp(test)


    matcher = Matcher(nlp.vocab)
    matcher.add("Noun", [[{"POS": "NOUN"},{"POS": "NOUN"}, {"POS": "NOUN"}]])

    matches = matcher(test)

    for i in range(0,len(matches)):
        token = test[matches[i][1]:matches[i][2]]
        #collect_all_matched_skill.append(str(token))
        if str(token).lower() not in exclude_item_list:
            #print(token)
            experience_word_list.append(str(token))

    matcher = Matcher(nlp.vocab)
    matcher.add("PROP & Noun", [[{"POS": "PROPN"}, {"POS": "NOUN"}]])

    matches = matcher(test)

    for i in range(0,len(matches)):
        token = test[matches[i][1]:matches[i][2]]
        #collect_all_matched_skill.append(str(token))
        if str(token).lower() not in exclude_item_list:
            #print(token)
            experience_word_list.append(str(token))
        
    matcher = Matcher(nlp.vocab)
    matcher.add("ADJ & Noun", [[{"POS": "ADJ"}, {"POS": "NOUN"}]])

    matches = matcher(test)

    for i in range(0,len(matches)):
        token = test[matches[i][1]:matches[i][2]]
        #collect_all_matched_skill.append(str(token))
        if str(token).lower() not in exclude_item_list:
            #print(token)
            experience_word_list.append(str(token).lower())
    
    return experience_word_list

def remove_stopwords(text):
    text_tokens = text.split(" ")
    tokens_filtered= [word for word in text_tokens if not word.lower() in stop_words]
    return (" ").join(tokens_filtered)


def check_sent(word, sentences): 
    final = [all([w in x for w in word]) for x in sentences] 
    sent_len = [sentences[i] for i in range(0, len(final)) if final[i]]
    return int(len(sent_len))



In [ ]:
def calculate_tfidf(text):
    doc = text.replace('\r', '')
    doc = doc.replace('\n', ' ')

    # brackets appear in all instances
    doc = doc.replace('[', '')
    doc = doc.replace(']', '')
    doc = doc.replace(')', '')
    doc = doc.replace('(', '')
    doc = doc.replace(',', '')
    doc = doc.replace('—', '')
    doc = doc.replace('-', '')

        
    total_words = doc.split()
    total_word_length = len(total_words)
    #print(total_word_length)
    total_sentences = tokenize.sent_tokenize(doc)
    total_sent_len = len(total_sentences)
    #print(total_sent_len)
    tf_score = {}
    for each_word in total_words:
        each_word = each_word.replace('.','')
        if each_word not in stop_words:
            if each_word in tf_score:
                tf_score[each_word] += 1
            else:
                tf_score[each_word] = 1

    # Dividing by total_word_length for each dictionary element
    tf_score.update((x, y/int(total_word_length)) for x, y in tf_score.items())
    #print(tf_score)
    
    idf_score = {}
    for each_word in total_words:
        each_word = each_word.replace('.','')
        if each_word not in stop_words:
            if each_word in idf_score:
                idf_score[each_word] = check_sent(each_word, total_sentences)
            else:
                idf_score[each_word] = 1

    # Performing a log and divide
    

    idf_score.update((x, math.log(int(total_sent_len)/y)) for x, y in idf_score.items())
    #print(idf_score)
    tf_idf_score = {key: tf_score[key] * idf_score.get(key, 0) for key in tf_score.keys()}
    #print(tf_idf_score)
    return tf_idf_score


def validate_list_by_tfidf(txt, lst):
    prob_dic = calculate_tfidf(txt)
    cleaned_list=[]
    test=lst
    test=list(set(test))

    updated_dic = dict((k.lower(), v) for k,v in prob_dic.items())

    chk=[k.lower()  for  k in  updated_dic.keys()]
    temp_dict={}
    total=0
    for words in test:
        #print(words)
        i = words.split(' ')

        sum=0

        for j in i:
            if j in chk:

                #print(j, updated_dic[j])
                sum+=updated_dic[j]
        #print(sum)
        temp_dict[words]=sum
        total+=sum
        #print()

    #print(total/len(test))
    limit = round(total/len(test), 4)
    #print(limit)
    
    for i,j in temp_dict.items():
        if j > limit:
            #print(i)
            cleaned_list.append(str(i))
        #else:
            #print('-------  ', i,j)
        
    return cleaned_list

In [ ]:
def hard_skill_patterns(text):
    collect_all_matched_skill=[]
    words_pos_dict={}
    doc = clean_text(text)
    doc = remove_stopwords(doc)
    doc = nlp(doc)

    
    for token in doc:
        #if not token.is_punct and not token.is_stop  and token.lower_ not in stop_words and token.lemma_ not in stop_words and token.pos_ != 'SPACE':
        #print(token, ' -> ', token.pos_, ' -> ', )
        if str(token) not in words_pos_dict:
            item = str(token)
            words_pos_dict[item]=token.pos_
            
    #pattern 
    
    matcher = Matcher(nlp.vocab)
    matcher.add("Noun", [[{"POS": "NOUN"},{"POS": "NOUN","OP": "+"}]])

    #doc = nlp(u'what are the main issues')
    matches = matcher(doc)

    for i in range(0,len(matches)):

        # match: id, start, end
        token = doc[matches[i][1]:matches[i][2]]
        # append token to list
        if str(token) not in collect_all_matched_skill:
            #print(token)
            collect_all_matched_skill.append(str(token).lower())

    #pattern 
    
    '''
    matcher = Matcher(nlp.vocab)
    matcher.add("PROP", [[{"POS": "PROPN"}]])

    #doc = nlp(u'what are the main issues')
    matches = matcher(doc)

    for i in range(0,len(matches)):

        # match: id, start, end
        token = doc[matches[i][1]:matches[i][2]]
        # append token to list
        if str(token) not in collect_all_matched_skill:
            #print(token)
            collect_all_matched_skill.append(str(token).lower())
    '''       
    #pattern 
    
    
    matcher = Matcher(nlp.vocab)
    matcher.add("PROP", [[{"POS": "PROPN"},{"POS": "PROPN","OP": "+"}]])

    #doc = nlp(u'what are the main issues')
    matches = matcher(doc)

    for i in range(0,len(matches)):

        # match: id, start, end
        token = doc[matches[i][1]:matches[i][2]]
        # append token to list
        if str(token) not in collect_all_matched_skill:
            #print(token)
            collect_all_matched_skill.append(str(token).lower())
    
            
    #pattern
    
    matcher = Matcher(nlp.vocab)
    matcher.add("PROP & Noun", [[{"POS": "PROPN"}, {"POS": "PROPN"}, {"POS": "NOUN"}]])

    matches = matcher(doc)

    for i in range(0,len(matches)):
        token = doc[matches[i][1]:matches[i][2]]
        if str(token) not in collect_all_matched_skill:
            #print(token)
            collect_all_matched_skill.append(str(token).lower())    
    
    #pattern 
    
    matcher = Matcher(nlp.vocab)
    matcher.add("PROP & Verb", [[{"POS": "PROPN"}, {"POS": "PROPN"}, {"POS": "VERB"}]])

    matches = matcher(doc)

    for i in range(0,len(matches)):
        token = doc[matches[i][1]:matches[i][2]]
        if str(token) not in collect_all_matched_skill:
            #print(token)
            collect_all_matched_skill.append(str(token).lower())    
            
    #pattern 
    
   
    
    matcher = Matcher(nlp.vocab)
    matcher.add("Verb & Noun", [[{"POS": "VERB"}, {"POS": "NOUN"}, {"POS": "NOUN"}]])

    matches = matcher(doc)

    for i in range(0,len(matches)):
        token = doc[matches[i][1]:matches[i][2]]
        if str(token) not in collect_all_matched_skill:
            #print(token)
            collect_all_matched_skill.append(str(token).lower()) 
    
    
    #pattern 
    matcher = Matcher(nlp.vocab)
    matcher.add("PROP", [[{"POS": "PROPN"},{"POS": "NOUN","OP": "+"}]])

    #doc = nlp(u'what are the main issues')
    matches = matcher(doc)

    for i in range(0,len(matches)):

        # match: id, start, end
        token = doc[matches[i][1]:matches[i][2]]
        # append token to list
        if str(token) not in collect_all_matched_skill:
            #print(token)
            collect_all_matched_skill.append(str(token).lower())
            
    return collect_all_matched_skill

In [ ]:
def soft_skill_patterns(text):
    collect_all_matched_skill=[]

    doc = clean_text(text)
    doc = remove_stopwords(doc)
    doc = nlp(doc)

    #pattern 1 
    
    matcher = Matcher(nlp.vocab)
    matcher.add("Noun", [[{"POS": "NOUN"}]])

    #doc = nlp(u'what are the main issues')
    matches = matcher(doc)

    for i in range(0,len(matches)):

        # match: id, start, end
        token = doc[matches[i][1]:matches[i][2]]
        # append token to list
        if str(token) not in collect_all_matched_skill:
            if not any(word in str(token) for word in exclude_item_list):

                #print(token)
                #collect_all_matched_skill.append(str(token))
                collect_all_matched_skill.append(str(token).lower()) 
    
    matcher = Matcher(nlp.vocab)
    matcher.add("Noun", [[{"POS": "PROPN"}]])

    #doc = nlp(u'what are the main issues')
    matches = matcher(doc)

    for i in range(0,len(matches)):

        # match: id, start, end
        token = doc[matches[i][1]:matches[i][2]]
        # append token to list
        if str(token) not in collect_all_matched_skill:
            if not any(word in str(token) for word in exclude_item_list):

                #print(token)
                #collect_all_matched_skill.append(str(token))
                collect_all_matched_skill.append(str(token).lower()) 
     
    
    #pattern 2
    
    
    matcher = Matcher(nlp.vocab)
    matcher.add("ADJ", [[{"POS": "ADJ"}]])

    #doc = nlp(u'what are the main issues')
    matches = matcher(doc)

    for i in range(0,len(matches)):

        # match: id, start, end
        token = doc[matches[i][1]:matches[i][2]]
        # append token to list
        if str(token) not in collect_all_matched_skill:
            if not any(word in str(token) for word in exclude_item_list):

                #print(token)
                #collect_all_matched_skill.append(str(token))
                collect_all_matched_skill.append(str(token).lower()) 
    
    #pattern 3
    
    matcher = Matcher(nlp.vocab)
    matcher.add("Verb & PROP & Noun", [[{"POS": "VERB"},{"POS": "PROPN"}, {"POS": "NOUN"}]])

    matches = matcher(doc)

    for i in range(0,len(matches)):
        token = doc[matches[i][1]:matches[i][2]]
        if str(token) not in collect_all_matched_skill:
            if not any(word in str(token) for word in exclude_item_list):

                #print(token)
                #collect_all_matched_skill.append(str(token))
                collect_all_matched_skill.append(str(token).lower()) 
            
    #pattern 4
    
    matcher = Matcher(nlp.vocab)
    matcher.add("Verb", [[{"POS": "VERB"}, {"POS": "VERB"}]])

    matches = matcher(doc)

    for i in range(0,len(matches)):
        token = doc[matches[i][1]:matches[i][2]]
        if str(token) not in collect_all_matched_skill:
            if not any(word in str(token) for word in exclude_item_list):

                #print(token)
                #collect_all_matched_skill.append(str(token))
                collect_all_matched_skill.append(str(token).lower())     
    
    #pattern 5
    
    matcher = Matcher(nlp.vocab)
    matcher.add("Verb & Noun", [[{"POS": "VERB"}, {"POS": "NOUN"}, {"POS": "NOUN"}]])

    matches = matcher(doc)

    for i in range(0,len(matches)):
        token = doc[matches[i][1]:matches[i][2]]
        if str(token) not in collect_all_matched_skill:
            if not any(word in str(token) for word in exclude_item_list):

                #print(token)
                #collect_all_matched_skill.append(str(token))
                collect_all_matched_skill.append(str(token).lower())     
            
    #pattern 6
    
    matcher = Matcher(nlp.vocab)
    matcher.add("ADJ & ADJ", [[{"POS": "ADJ"}, {"POS": "ADJ"}]])

    matches = matcher(doc)

    for i in range(0,len(matches)):
        token = doc[matches[i][1]:matches[i][2]]
        if str(token) not in collect_all_matched_skill:
            if not any(word in str(token) for word in exclude_item_list):

                #print(token)
                #collect_all_matched_skill.append(str(token))
                collect_all_matched_skill.append(str(token).lower()) 
    
    #pattern 7 
    matcher = Matcher(nlp.vocab)
    matcher.add("Noun & Verb", [[{"POS": "NOUN"},{"POS": "VERB"},{"POS": "NOUN"}]])

    matches = matcher(doc)

    for i in range(0,len(matches)):
        token = doc[matches[i][1]:matches[i][2]]
        if str(token) not in collect_all_matched_skill:
            if not any(word in str(token) for word in exclude_item_list):

                #print(token)
                #collect_all_matched_skill.append(str(token))
                collect_all_matched_skill.append(str(token).lower()) 
            
            
    #pattern 8
    matcher = Matcher(nlp.vocab)
    matcher.add("Adj & Noun", [[{"POS": "ADJ"}, {"POS": "NOUN"},{"POS": "NOUN"}]])

    matches = matcher(doc)

    for i in range(0,len(matches)):
        token = doc[matches[i][1]:matches[i][2]]
        if str(token) not in collect_all_matched_skill:
            if not any(word in str(token) for word in exclude_item_list):

                #print(token)
                #collect_all_matched_skill.append(str(token))
                collect_all_matched_skill.append(str(token).lower()) 
            
    #pattern 9
    matcher = Matcher(nlp.vocab)
    matcher.add("Adj & Verb", [[{"POS": "ADJ"}, {"POS": "VERB"}]])

    matches = matcher(doc)

    for i in range(0,len(matches)):
        token = doc[matches[i][1]:matches[i][2]]
        if str(token) not in collect_all_matched_skill:
            if not any(word in str(token) for word in exclude_item_list):

                #print(token)
                collect_all_matched_skill.append(str(token).lower()) 
            
            
    #pattern 11
    matcher = Matcher(nlp.vocab)
    matcher.add("Adj & Noun", [[{"POS": "VERB"},{"POS": "ADJ"}, {"POS": "NOUN"},{"POS": "NOUN"}]])

    matches = matcher(doc)

    for i in range(0,len(matches)):
        token = doc[matches[i][1]:matches[i][2]]
        if str(token) not in collect_all_matched_skill:
            if not any(word in str(token) for word in exclude_item_list):

                #print(token)
                #collect_all_matched_skill.append(str(token))
                collect_all_matched_skill.append(str(token).lower()) 
                
    #pattern 12
    matcher = Matcher(nlp.vocab)
    matcher.add("Noun & Verb", [[{"POS": "NOUN"},{"POS": "NOUN"},{"POS": "VERB"}]])

    matches = matcher(doc)

    for i in range(0,len(matches)):
        token = doc[matches[i][1]:matches[i][2]]
        if str(token) not in collect_all_matched_skill:
            if not any(word in str(token) for word in exclude_item_list):

                #print(token)
                #collect_all_matched_skill.append(str(token))
                collect_all_matched_skill.append(str(token).lower()) 
                
                
    #pattern 13
    matcher = Matcher(nlp.vocab)
    matcher.add("Verb & Noun", [[{"POS": "VERB"},{"POS": "NOUN"}]])

    matches = matcher(doc)

    for i in range(0,len(matches)):
        token = doc[matches[i][1]:matches[i][2]]
        if str(token) not in collect_all_matched_skill:
            if not any(word in str(token) for word in exclude_item_list):

                #print(token)
                #collect_all_matched_skill.append(str(token))
                collect_all_matched_skill.append(str(token).lower()) 
                
    #pattern 14
    matcher = Matcher(nlp.vocab)
    matcher.add("Adj & Noun", [[{"POS": "ADJ"}, {"POS": "VERB"},{"POS": "NOUN"}]])

    matches = matcher(doc)

    for i in range(0,len(matches)):
        token = doc[matches[i][1]:matches[i][2]]
        if str(token) not in collect_all_matched_skill:
            if not any(word in str(token) for word in exclude_item_list):

                #print(token)
                #collect_all_matched_skill.append(str(token))
                collect_all_matched_skill.append(str(token).lower())  
                
                
    #pattern 15
    matcher = Matcher(nlp.vocab)
    matcher.add("Adj & Noun", [[{"POS": "ADJ"},{"POS": "NOUN"}]])

    matches = matcher(doc)

    for i in range(0,len(matches)):
        token = doc[matches[i][1]:matches[i][2]]
        if str(token) not in collect_all_matched_skill:
            if not any(word in str(token) for word in exclude_item_list):

                #print(token)
                #collect_all_matched_skill.append(str(token))
                collect_all_matched_skill.append(str(token).lower())  
            
            
    #pattern 16
    matcher = Matcher(nlp.vocab)
    matcher.add("PROPN & Noun", [[{"POS": "PROPN"},{"POS": "NOUN"}]])

    matches = matcher(doc)
    
    
    for i in range(0,len(matches)):
        token = doc[matches[i][1]:matches[i][2]]
        
        if str(token) not in collect_all_matched_skill:
            if not any(word in str(token) for word in exclude_item_list):

                #print(token)
                #collect_all_matched_skill.append(str(token))
                collect_all_matched_skill.append(str(token).lower())  
        
    #print()        
    #pattern 17
    matcher = Matcher(nlp.vocab)
    matcher.add("Noun", [[{"POS": "NOUN"},{"POS": "VERB"}]])

    #doc = nlp(u'what are the main issues')
    matches = matcher(doc)

    for i in range(0,len(matches)):

        # match: id, start, end
        token = doc[matches[i][1]:matches[i][2]]
        # append token to list
        if str(token) not in collect_all_matched_skill:
            if not any(word in str(token) for word in exclude_item_list):

                #print(token)
                #collect_all_matched_skill.append(str(token))
                collect_all_matched_skill.append(str(token).lower()) 
    
    return collect_all_matched_skill

In [ ]:
f1="Job 1, Risk Management Financial Services.docx"
f2="Job 2, Sports Phsiotherapist.docx"
f3="Job 3, Retail Branch Manager.docx"
f4="Job 4, Secondary School Teacher.docx"
f5="Job 5, Senior Marketing Manager.docx"

skills=main(f5)
print(skills)

--------Soft Skills-----------
{'active users', 'cultures team player', 'relative key', 'sexual orientation', 'marketing', 'influencing skills', 'required willingness travel', 'ability launch', 'proven ability', 'high attention detail', 'making decisions', 'team player', 'complex decisions', 'key accountabilities', 'negotiation influencing skills', 'eager learn', 'willing partner'}
--------Hard Skills-----------
{'dress schedule', 'research papers', 'market rate salary', 'leaders stakeholders', 'identity framework', 'job types', 'competitor intelligence', 'china singapore', 'empowers sales channels', 'gender identity', 'team people', 'strategies product launches', 'team player', 'core archetypes', 'business acumen', 'cov guidelines', 'stakeholder relationships', 'authority marketing effectiveness', 'developing product marketing', 'case studies', 'united kingdom'}
{
    "Soft_skils": [
        "active users",
        "negotiation influencing skills",
        "cultures team player",
    